In [1]:
import theano
import theano.tensor as T
import numpy as np

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: no CUDA-capable device is detected)


In [2]:
DIMENSIONS = 5

In [3]:
v = theano.shared( np.empty(DIMENSIONS, dtype=np.float32), name="v")
X = T.fmatrix(name='X')

In [4]:
X_dot_v = T.dot(X,v)
theano.pp(X_dot_v)

'(X \\dot v)'

In [5]:
objectiveFunction = T.dot(X_dot_v.T, X_dot_v)
theano.pp(objectiveFunction)

'((X \\dot v) \\dot (X \\dot v))'

In [6]:
grad_v = T.grad(objectiveFunction, v)
theano.pp(grad_v)

'(X.T \\dot ((fill(((X \\dot v) \\dot (X \\dot v)), TensorConstant{1.0}) * (X \\dot v)) + (fill(((X \\dot v) \\dot (X \\dot v)), TensorConstant{1.0}) * (X \\dot v))))'

In [7]:
eta = 0.5

In [8]:
y = v + eta * grad_v
updateFunction = y / y.norm(2)
print theano.pp(y)
print theano.pp(updateFunction)

(v + (TensorConstant{0.5} * (X.T \dot ((fill(((X \dot v) \dot (X \dot v)), TensorConstant{1.0}) * (X \dot v)) + (fill(((X \dot v) \dot (X \dot v)), TensorConstant{1.0}) * (X \dot v))))))
((v + (TensorConstant{0.5} * (X.T \dot ((fill(((X \dot v) \dot (X \dot v)), TensorConstant{1.0}) * (X \dot v)) + (fill(((X \dot v) \dot (X \dot v)), TensorConstant{1.0}) * (X \dot v)))))) / (Sum{acc_dtype=float64}((|(v + (TensorConstant{0.5} * (X.T \dot ((fill(((X \dot v) \dot (X \dot v)), TensorConstant{1.0}) * (X \dot v)) + (fill(((X \dot v) \dot (X \dot v)), TensorConstant{1.0}) * (X \dot v))))))| ** TensorConstant{2})) ** TensorConstant{0.5}))


In [9]:
train = theano.function(
    inputs=[X],
    outputs=updateFunction,
    updates=((v, updateFunction),)
)

In [10]:
def getFirstEigenValueAndVector(X_data, epsilon=1e-100, max_iteraions = 20):
    v.set_value(np.random.rand(DIMENSIONS).astype('float32'))
    print 'Getting eigen values for: X = \n', X_data
    # print 'Starting with a random vector\n', v.get_value()
    last_v_star = v.get_value()
    for i in range(max_iteraions):
        v_star = train(X_data)
        rms = sum(v_star - last_v_star)**2
        print 'rms difference: ', rms
        last_v_star = v_star
        if(rms < epsilon):
            break
    w_star = objectiveFunction.eval({X:X_data})
    print "Eigen Value:", w_star
    print "Eigen Vector:\n", v_star
    return w_star, v_star

In [11]:
# X_data = np.array([[ (i+1)*(j+1) for j in range(DIMENSIONS)] for i in range(DIMENSIONS)]).astype('float32')
X_data = np.random.rand(DIMENSIONS,DIMENSIONS).astype('float32')
epsilon = 1e-100
max_iteraions = 20

In [12]:
print v
# XTX = np.dot(X_data.T,X_data)
w_star, v_star = getFirstEigenValueAndVector( X_data )

v
Getting eigen values for: X = 
[[ 0.23975161  0.35725662  0.959858    0.38111109  0.04587001]
 [ 0.46347278  0.55025983  0.71960729  0.03448376  0.41214687]
 [ 0.33262357  0.3759295   0.01549499  0.89723474  0.16308531]
 [ 0.37765834  0.29537737  0.7302109   0.59562129  0.69434047]
 [ 0.32382259  0.87839311  0.22292301  0.44921654  0.20482525]]
rms difference:  0.089348519542
rms difference:  0.000722918760844
rms difference:  2.48499743662e-05
rms difference:  1.11510577572e-06
rms difference:  5.56700365806e-08
rms difference:  2.99721936159e-09
rms difference:  1.71951342054e-10
rms difference:  1.19513288155e-11
rms difference:  6.47482067961e-13
rms difference:  4.35207425653e-14
rms difference:  1.42108547152e-14
rms difference:  0.0
Eigen Value: 4.86833953857
Eigen Vector:
[ 0.35099527  0.48652834  0.56604934  0.46010408  0.32861057]


In [13]:
# v_star_mat = np.matrix([v_star]).T
# XTX - w_star * np.dot(v_star_mat, v_star_mat.T)
# getFirstEigenValueAndVector(XTX - w_star * np.dot(v_star_mat, v_star_mat.T))

In [14]:
# w_np,v_np = np.linalg.eigh( np.dot(X_data.T,X_data) )
print X_data
w_np,v_np = np.linalg.eigh(np.dot(X_data.T,X_data))
idx = np.argsort(-w_np)
w_np = w_np[idx]
v_np = v_np[:,idx]
for i in range(DIMENSIONS):
    print w_np[i]
    print v_np[:,i]

[[ 0.23975161  0.35725662  0.959858    0.38111109  0.04587001]
 [ 0.46347278  0.55025983  0.71960729  0.03448376  0.41214687]
 [ 0.33262357  0.3759295   0.01549499  0.89723474  0.16308531]
 [ 0.37765834  0.29537737  0.7302109   0.59562129  0.69434047]
 [ 0.32382259  0.87839311  0.22292301  0.44921654  0.20482525]]
4.86834
[-0.35099524 -0.48652837 -0.56604934 -0.46010408 -0.32861057]
0.826942
[ 0.04309028  0.2212525  -0.71164477  0.66130418 -0.07368239]
0.370609
[-0.10008702 -0.79745758  0.19837654  0.51227647  0.2286122 ]
0.228333
[-0.20004645  0.04738816  0.35582745  0.28355584 -0.86644024]
0.0197107
[-0.90824533  0.27595881  0.08475541  0.09027689  0.28914317]
